In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, silhouette_samples, silhouette_score
plt.style.use('fivethirtyeight')
%matplotlib inline

In [2]:
insurance = pd.read_csv('insurance_part2_data.csv')
insurance.head()

,Age,Agency_Code,Type,Claimed,Commision,Channel,Duration,Sales,Product Name,Destination
0,48,C2B,Airlines,No,0.70,Online,7,2.51,Customised Plan,ASIA
1,36,EPX,Travel Agency,No,0.00,Online,34,20.00,Customised Plan,ASIA
2,39,CWT,Travel Agency,No,5.94,Online,3,9.90,Customised Plan,Americas
3,36,EPX,Travel Agency,No,0.00,Online,4,26.00,Cancellation Plan,ASIA
4,33,JZI,Airlines,No,6.30,Online,53,18.00,Bronze Plan,ASIA


In [3]:
insurance.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Age,3000,NaN,NaN,NaN,38.091,10.4635,8,32,36,42,84
Agency_Code,3000,4,EPX,1365,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Type,3000,2,Travel Agency,1837,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Claimed,3000,2,No,2076,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Commision,3000,NaN,NaN,NaN,14.5292,25.4815,0,0,4.63,17.235,210.21
Channel,3000,2,Online,2954,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Duration,3000,NaN,NaN,NaN,70.0013,134.053,-1,11,26.5,63,4580
Sales,3000,NaN,NaN,NaN,60.2499,70.734,0,20,33,69,539
Product Name,3000,5,Customised Plan,1136,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Destination,3000,3,ASIA,2465,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
insurance[(insurance.Sales == 0) | (insurance.Commision == 0)]

,Age,Agency_Code,Type,Claimed,Commision,Channel,Duration,Sales,Product Name,Destination
1,36,EPX,Travel Agency,No,0.0,Online,34,20.0,Customised Plan,ASIA
3,36,EPX,Travel Agency,No,0.0,Online,4,26.0,Cancellation Plan,ASIA
7,36,EPX,Travel Agency,No,0.0,Online,16,80.0,Cancellation Plan,ASIA
8,36,EPX,Travel Agency,No,0.0,Online,19,14.0,Cancellation Plan,ASIA
9,36,EPX,Travel Agency,No,0.0,Online,42,43.0,Cancellation Plan,ASIA
...,...,...,...,...,...,...,...,...,...,...
2988,36,EPX,Travel Agency,Yes,0.0,Online,18,36.0,Customised Plan,ASIA
2989,35,EPX,Travel Agency,No,0.0,Online,26,20.0,Customised Plan,ASIA
2990,51,EPX,Travel Agency,No,0.0,Online,2,20.0,Customised Plan,ASIA
2993,36,EPX,Travel Agency,No,0.0,Online,234,10.0,Cancellation Plan,ASIA


In [5]:
insurance[(insurance.Sales == 0)]

,Age,Agency_Code,Type,Claimed,Commision,Channel,Duration,Sales,Product Name,Destination
131,53,JZI,Airlines,No,12.95,Online,93,0.0,Bronze Plan,ASIA
162,36,EPX,Travel Agency,No,0.00,Online,2,0.0,Customised Plan,ASIA
323,54,CWT,Travel Agency,No,100.98,Online,18,0.0,Customised Plan,Americas
483,44,CWT,Travel Agency,No,11.88,Online,10,0.0,Customised Plan,ASIA
513,31,CWT,Travel Agency,No,83.16,Online,99,0.0,Customised Plan,EUROPE
537,34,CWT,Travel Agency,No,11.88,Online,45,0.0,Customised Plan,ASIA
612,33,CWT,Travel Agency,No,17.82,Online,4,0.0,Customised Plan,ASIA
646,31,CWT,Travel Agency,No,11.88,Online,144,0.0,Customised Plan,ASIA
703,30,JZI,Airlines,No,7.70,Online,9,0.0,Bronze Plan,ASIA
766,36,EPX,Travel Agency,No,0.00,Online,7,0.0,Cancellation Plan,ASIA


In [6]:
insurance.duplicated().sum()

139

In [8]:
round((139/3000)*100,2)

4.63

In [9]:
insurance[insurance.duplicated()]

,Age,Agency_Code,Type,Claimed,Commision,Channel,Duration,Sales,Product Name,Destination
63,30,C2B,Airlines,Yes,15.0,Online,27,60.0,Bronze Plan,ASIA
329,36,EPX,Travel Agency,No,0.0,Online,5,20.0,Customised Plan,ASIA
407,36,EPX,Travel Agency,No,0.0,Online,11,19.0,Cancellation Plan,ASIA
411,35,EPX,Travel Agency,No,0.0,Online,2,20.0,Customised Plan,ASIA
422,36,EPX,Travel Agency,No,0.0,Online,5,20.0,Customised Plan,ASIA
...,...,...,...,...,...,...,...,...,...,...
2940,36,EPX,Travel Agency,No,0.0,Online,8,10.0,Cancellation Plan,ASIA
2947,36,EPX,Travel Agency,No,0.0,Online,10,28.0,Customised Plan,ASIA
2952,36,EPX,Travel Agency,No,0.0,Online,2,10.0,Cancellation Plan,ASIA
2962,36,EPX,Travel Agency,No,0.0,Online,4,20.0,Customised Plan,ASIA


In [10]:
insurance1 = insurance.copy()

In [11]:
insurance1.drop_duplicates(inplace=True)
insurance1.shape

(2861, 10)

In [14]:
for col in insurance.columns:
    if insurance[col].dtype.name == 'object':
        insurance[col] = pd.Categorical(insurance[col]).codes

In [15]:
X = insurance.drop(['Claimed'], axis = 1)
y = insurance['Claimed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 50)

In [16]:
sc = StandardScaler()
X_train_ann = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
X_test_ann = pd.DataFrame(sc.transform(X_test), columns=X_test.columns)

In [17]:
params = { 'hidden_layer_sizes': tuple(np.arange(100,900,100)),
           'activation': ['relu', 'logistic'],
           'solver': ['sgd', 'adam'],
           'tol': [0.000001,0.01, 0.1],
           'max_iter': np.arange(1000,4000,1000)
        }

In [ ]:
ann_model = MLPClassifier()
grid_cv = GridSearchCV(estimator=ann_model, param_grid=params, cv=4)
grid_cv.fit(X_train_ann, y_train)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv